In [12]:
import pandas as pd
from sqlalchemy import create_engine

# Store CSV into Dataframe 

In [36]:
eruptions = "Resources/eruptions.csv"
eruptions_data = pd.read_csv(eruptions)

events = "Resources/events.csv"
events_df = pd.read_csv(events)

volcano = "Resources/volcano.csv"
volcano_df = pd.read_csv(volcano)

damage = "Resources/damage.csv"
damage_df = pd.read_csv(damage,sep=";")



In [38]:
volcano_df.head()

,volcano_number,volcano_name,primary_volcano_type,last_eruption_year,country,region,subregion,latitude,longitude,elevation,...,major_rock_5,minor_rock_1,minor_rock_2,minor_rock_3,minor_rock_4,minor_rock_5,population_within_5_km,population_within_10_km,population_within_30_km,population_within_100_km
0,283001,Abu,Shield(s),-6850,Japan,"Japan, Taiwan, Marianas",Honshu,34.500,131.600,641,...,,,,,,,3597,9594,117805,4071152
1,355096,Acamarachi,Stratovolcano,Unknown,Chile,South America,"Northern Chile, Bolivia and Argentina",-23.292,-67.618,6023,...,,,,,,,0,7,294,9092
2,342080,Acatenango,Stratovolcano(es),1972,Guatemala,México and Central America,Guatemala,14.501,-90.876,3976,...,,Basalt / Picro-Basalt,,,,,4329,60730,1042836,7634778
3,213004,Acigol-Nevsehir,Caldera,-2080,Turkey,Mediterranean and Western Asia,Turkey,38.537,34.621,1683,...,,,,,,,127863,127863,218469,2253483
4,321040,Adams,Stratovolcano,950,United States,Canada and Western USA,USA (Washington),46.206,-121.490,3742,...,,Dacite,,,,,0,70,4019,393303


In [ ]:
# getting data from 1800 onwards 
volcano_df2 =volcano_df.loc[volcano_df2.]